In [0]:
!pip install allennlp

     |████████████████████████████████| 7.6MB 3.1MB/s 
     |████████████████████████████████| 61kB 9.2MB/s 
     |████████████████████████████████| 163kB 54.2MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
     |████████████████████████████████| 204kB 51.6MB/s 
     |████████████████████████████████| 133kB 59.2MB/s 
     |████████████████████████████████| 256kB 49.4MB/s 
     |████████████████████████████████| 245kB 51.0MB/s 
     |████████████████████████████████| 1.0MB 58.2MB/s 
  Created wheel for ftfy: filename=ftfy-5.7-cp36-none-any.whl size=44593 sha256=7de1ad9ab9bad9dfa87b60136d687a4378d7289d7d9b4f0640660e3826da9622
  Stored in directory: /root/.cache/pip/wheels/8e/da/59/6c8925d571aacade638a0f515960c21c0887af1bfe31908fbf
  Created wheel for word2number: filename=word2number-1.1-cp36-none-any.whl size=5587 sha256=4c4ed944504bddee7c4092e7b47b712280a3fcd127f60bc2f473baa93a380bbe
  Stored in directory: /root/.cache/pip/wheels/46/2f/53/5f5c1d275492f2fce1cdab9a9bb12d49286

In [0]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_20newsgroups

import torch as tt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torchtext import data

import re

In [0]:
from allennlp.modules.elmo import Elmo

options_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"

elmo = Elmo(options_file, weight_file, 2, dropout=0)

100%|██████████| 374434792/374434792 [00:08<00:00, 44401828.66B/s]


In [0]:
#device = tt.device('cuda') if tt.cuda.is_available() else tt.device('cpu')

In [0]:
import spacy
from spacy.symbols import ORTH

spacy_en = spacy.load('en')
spacy_en.remove_pipe('tagger')
spacy_en.remove_pipe('ner')

spacy_en.tokenizer.add_special_case("don't", [{ORTH: "do"}, {ORTH: "n't"}])

In [0]:
def tokenizer(text): 
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [0]:
def get_news(text):
  text = re.sub(r'([<| ].*?(\.edu)|(\.com)|(\.se)(>)?)', '', text)
  text = re.sub('\W+', ' ', text)
  return text

In [0]:
news = fetch_20newsgroups(subset='all')

In [0]:
df = pd.Series(news.data)
df = pd.DataFrame(df) 
df.columns = ['Data'] + df.columns.tolist()[1:]
df['target'] = pd.Series(news.target)

In [0]:
df['Data'] = df.apply(lambda row: get_news(row['Data']), axis=1)

In [0]:
df.head()

,Data,target
0,From Subject Pens fans reactions Organization ...,10
1,From Matthew B Lawson Subject Which high perfo...,3
2,From hilmi er dsv su Hilmi Eren Subject Re ARM...,17
3,From guyd austin ibm Guy Dawson Subject Re IDE...,3
4,From Subject driver Organization Sophomore Mec...,4


In [0]:
train, test, y_train, y_test = train_test_split(df.Data.values, df.target.values, test_size=0.2, random_state=42)

train, valid, y_train, y_valid = train_test_split(train, y_train, test_size=0.2, random_state=42)

In [0]:
#X_train = batch_to_ids(train.Data.values)
#y_train = tt.from_numpy(df_train.target.values).float()

#X_valid = batch_to_ids(valid.Data.values)
#y_valid = tt.from_numpy(valid.target.values).float()

#X_test = batch_to_ids(test.Data.values)
#y_test = tt.from_numpy(test.target.values).float()

In [0]:
def get_ids(data):
    word2id = {}
    id2word = {}
    i = 1
    max_len = 0
    
    for dtype in data:
        for comment in dtype:
            tokens = tokenizer(comment)
            if len(tokens) > max_len:
                max_len = len(tokens)
            
            for t in tokens:
                if not t in word2id:
                    word2id[t] = i
                    id2word[i] = t
                    i += 1
                    
    return word2id, id2word, max_len

In [0]:
def batch_to_ids(dtype, word2id, max_len):
    list_of_ids = []
    
    for comment in tqdm(dtype):
        comment_ids = []
        tokens = tokenizer(comment)
        
        for t in tokens:
            comment_ids.append(word2id[t])
        while len(comment_ids) < max_len:
            comment_ids.append(0)
        list_of_ids.append(comment_ids)
        
    return list_of_ids

In [0]:
word2id, id2word, max_len = get_ids((train, test, valid))

In [0]:
train_id = batch_to_ids(train, word2id, max_len)
valid_id = batch_to_ids(valid, word2id, max_len)
test_id = batch_to_ids(test, word2id, max_len)

100%|██████████| 3770/3770 [00:31<00:00, 118.69it/s]


In [0]:
def triplets(data, y):
    positive = []
    negative = []
    for i, comment in enumerate(data):
        positive.append(data[np.random.choice(np.where(y == y[i])[0])])
        negative.append(data[np.random.choice(np.where(y != y[i])[0])])
    
    return tt.tensor(positive), tt.tensor(negative)

In [0]:
tt.cuda.empty_cache()

batch_size = 32

pos, neg = triplets(test_id, y_test)
test_loader = DataLoader(TensorDataset(tt.tensor(test_id), pos, neg), batch_size=batch_size, drop_last=True)

pos, neg = triplets(train_id, y_train)
train_loader = DataLoader(TensorDataset(tt.tensor(train_id), pos, neg), batch_size=batch_size, drop_last=True)

pos, neg = triplets(valid_id, y_valid)
valid_loader = DataLoader(TensorDataset(tt.tensor(valid_id), pos, neg), batch_size=batch_size, drop_last=True)

In [0]:
def triplet_loss(anchor_embed, pos_embed, neg_embed):
    return F.cosine_similarity(anchor_embed, neg_embed) - F.cosine_similarity(anchor_embed, pos_embed)


class Tripletnet(nn.Module): 
    def __init__(self, vocab_size, embed_size, criterion, n_classes):
        super(Tripletnet, self).__init__()
        self.n_classes = n_classes
        self.embedding = nn.Embedding(vocab_size, embed_size)
        #self.elmo = elmo
        self.criterion = criterion
        #self.relu = nn. ReLU()
        #self.fc = nn.Linear(1024*2, 128)                          
        self.fc = nn.Sequential(nn.Linear(embed_size, 128*3),
                                nn.ReLU(),
                                nn.Linear(128*3, 128*3),
                                nn.ReLU(),
                                nn.Linear(128*3, n_classes)
                                )
        #self.out = nn.Linear(128*3, n_classes)

    def branch(self, x):
        x = self.embedding(x)
        x = x.mean(dim=1)
        #x = self.elmo(x)
        #x = tt.cat(x, dim=-1)
        #x = x.mean(dim=1)
        x = self.fc(x)

        return x       
        
    def forward(self, batch):     
        anchor = self.branch(batch[0])
        pos = self.branch(batch[1])
        neg = self.branch(batch[2])       
        
        return triplet_loss(anchor, pos, neg)


In [0]:
def _train_epoch(model, iterator, optimizer, curr_epoch):

    model.train()

    running_loss = 0

    n_batches = len(iterator)
    iterator = tqdm(iterator, total=n_batches, desc='epoch %d' % (curr_epoch), leave=True)

    for i, batch in enumerate(iterator):
        optimizer.zero_grad()

        loss = model(batch)
        loss = tt.abs(tt.mean(loss))
        #loss.mean().backward()
        optimizer.step()

        curr_loss = loss.data.cpu().detach().item()
        
        loss_smoothing = i / (i+1)
        running_loss = loss_smoothing * running_loss + (1 - loss_smoothing) * curr_loss

        iterator.set_postfix(loss='%.5f' % running_loss)

    return running_loss

def _test_epoch(model, iterator):
    model.eval()
    epoch_loss = 0

    n_batches = len(iterator)
    with tt.no_grad():
        for batch in iterator:
            loss = model(batch)
            loss = tt.abs(tt.mean(loss))
            epoch_loss += loss.data.item()

    return epoch_loss / n_batches


def nn_train(model, train_iterator, valid_iterator, optimizer, n_epochs=100,
          scheduler=None, early_stopping=0):

    prev_loss = 100500
    es_epochs = 0
    best_epoch = None
    history = pd.DataFrame()

    for epoch in range(n_epochs):
        train_loss = _train_epoch(model, train_iterator, optimizer, epoch)
        valid_loss = _test_epoch(model, valid_iterator)

        valid_loss = valid_loss
        print('validation loss %.5f' % valid_loss)

        record = {'epoch': epoch, 'train_loss': train_loss, 'valid_loss': valid_loss}
        history = history.append(record, ignore_index=True)

        if early_stopping > 0:
            if valid_loss > prev_loss:
                es_epochs += 1
            else:
                es_epochs = 0

            if es_epochs >= early_stopping:
                best_epoch = history[history.valid_loss == history.valid_loss.min()].iloc[0]
                print('Early stopping! best epoch: %d val %.5f' % (best_epoch['epoch'], best_epoch['valid_loss']))
                break

            prev_loss = min(prev_loss, valid_loss)

In [0]:
n_classes = df.target.unique().shape[0]

In [0]:
n_classes

20

In [0]:
tt.cuda.empty_cache()

model = Tripletnet(len(word2id)+1, 512, nn.BCEWithLogitsLoss(), n_classes)
#model = Tripletnet(elmo, nn.BCEWithLogitsLoss(), n_classes)
optimizer = optim.Adam(model.parameters(), lr=0.01)
#model.to(device)

In [0]:
nn_train(model, train_loader, valid_loader, optimizer, n_epochs=2)

epoch 1:   0%|          | 0/376 [00:00<?, ?it/s]

validation loss 0.00001


epoch 1: 100%|██████████| 376/376 [02:51<00:00,  2.20it/s, loss=0.00003]


validation loss 0.00001


In [0]:
for instance in list(tqdm._instances): 
    tqdm._decr_instances(instance)